# 1_ Import dependencies

In [17]:
# import env libraries
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

# import classic libraries
import numpy as np
import random
import os

# import RL model libraries
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy


# 2_ Type of spaces

In [18]:
Discrete(4).sample() # gives a values 0, 1, 2 and 3

3

In [19]:
Box(0,1,shape=(3,3)).sample() # Matrice 3*3 de valeurs entre 0 et 1

array([[0.22474548, 0.9805629 , 0.13530807],
       [0.67029667, 0.5322669 , 0.9068248 ],
       [0.3361421 , 0.4785081 , 0.41423726]], dtype=float32)

In [20]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[204, 250, 221],
       [222,  24, 242],
       [217,  71,  88]])

In [21]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample() # Tuple formé d'un valeur Discrete et d'une box

(1, array([21.248789], dtype=float32))

In [22]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample() # Dictionnaire d'éléments spaces

OrderedDict([('height', 1), ('speed', array([66.59647], dtype=float32))])

In [23]:
MultiBinary(4).sample()

array([0, 1, 0, 0], dtype=int8)

In [24]:
MultiDiscrete([5,2,2]).sample()

array([3, 1, 0], dtype=int64)

# 3_ Build an env

In [25]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [26]:
env=ShowerEnv()

In [27]:
env.observation_space.sample()

array([64.22858], dtype=float32)

In [28]:
env.reset()

array([38.])

# 4_ Test Env

In [29]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-58
Episode:2 Score:-60
Episode:3 Score:-40
Episode:4 Score:-16
Episode:5 Score:-60


# 5_ Train model

In [30]:
log_path = os.path.join('training', 'logs')

In [31]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [33]:
model.learn(total_timesteps=100000)

Logging to training\logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 21.4     |
| time/              |          |
|    fps             | 1315     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 23.9         |
| time/                   |              |
|    fps                  | 839          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0063989884 |
|    clip_fraction        | 0.172        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.953       |
|    explained_variance   |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 45.4        |
| time/                   |             |
|    fps                  | 635         |
|    iterations           | 11          |
|    time_elapsed         | 35          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.005710303 |
|    clip_fraction        | 0.139       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.826      |
|    explained_variance   | -3.72e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 40.1        |
|    n_updates            | 430         |
|    policy_gradient_loss | 0.00226     |
|    value_loss           | 80.8        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 49.1       |
| time/                   |            |
|    fps                  | 627        |
|    iterations           | 21         |
|    time_elapsed         | 68         |
|    total_timesteps      | 43008      |
| train/                  |            |
|    approx_kl            | 0.01618551 |
|    clip_fraction        | 0.151      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.877     |
|    explained_variance   | -3.58e-06  |
|    learning_rate        | 0.0003     |
|    loss                 | 50         |
|    n_updates            | 530        |
|    policy_gradient_loss | 0.00734    |
|    value_loss           | 95         |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 52.6        |
| time/                   |             |
|    fps                  | 619         |
|    iterations           | 31          |
|    time_elapsed         | 102         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.016179107 |
|    clip_fraction        | 0.169       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.832      |
|    explained_variance   | 4.17e-07    |
|    learning_rate        | 0.0003      |
|    loss                 | 56.8        |
|    n_updates            | 630         |
|    policy_gradient_loss | 0.00863     |
|    value_loss           | 98.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 54.3        |
| time/                   |             |
|    fps                  | 598         |
|    iterations           | 41          |
|    time_elapsed         | 140         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.013538335 |
|    clip_fraction        | 0.151       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.744      |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 44.5        |
|    n_updates            | 730         |
|    policy_gradient_loss | 0.0108      |
|    value_loss           | 106         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

# 6_ Save model

In [34]:
model.save('PPO')

In [36]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(11.8, 58.62729739634943)